In [1]:
import pathlib
import sys

In [2]:
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm

In [3]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
interim_dir_path = data_dir_path / "interim"
interim_path = interim_dir_path / "interim.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"
prediction_path = models_dir_path / "prediction.parquet"
src_dir_path = root_dir_path / "src"

In [4]:
sys.path.append(str(src_dir_path))

In [5]:
from package.constants import *
from package.feature_extraction import *

In [6]:
max_lags = 56

In [7]:
interim = pd.read_parquet(interim_path)

In [8]:
model = joblib.load(model_path)

In [9]:
is_test = interim["demand"].isnull()
dates = interim.loc[is_test, "date"].unique()

In [10]:
for date in tqdm(dates):
    is_date = (interim["date"] >= date - np.timedelta64(max_lags, "D")) & (
        interim["date"] <= date
    )

    intermediate = interim[is_date].copy()

    extract_features(intermediate)

    is_date = intermediate["date"] == date
    X = intermediate.loc[is_date, features]
    y_pred = model.predict(X)
    y_pred = np.maximum(0.0, y_pred)

    is_date = interim["date"] == date
    interim.loc[is_date, "demand"] = y_pred

100%|██████████| 56/56 [1:17:55<00:00, 83.50s/it]


In [11]:
interim[is_test].to_parquet(prediction_path)